Create a combined Dataframe of price and load for the years 2023-2025

In [ ]:
import pandas as pd
from datetime import datetime
import os

def price_data():
    folder_path = "C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data"
    file_names = [
        "Energy_Prices_2023.xlsx",
        "Energy_Prices_2024.xlsx",
        "Energy_Prices_2025.xlsx"
    ]

    def load_and_process_excel(file_path):
        filename = os.path.basename(file_path)
        year_in_file = ''.join(filter(str.isdigit, filename))
        
        df = pd.read_excel(file_path, skiprows=6, names=["Timestamp", "Price"])    
        df["Timestamp"] = df["Timestamp"].str.replace(r"\s*\(.*?\)", "", regex=True).str.strip()
        
        df.loc[df['Timestamp'] == '26/03/2023 01:00:00 - 26/03/2023 03:00:00', 'Timestamp'] = '26/03/2023 01:00:00 - 26/03/2023 02:00:00'
        df.loc[df['Timestamp'] == '31/03/2024 01:00:00 - 31/03/2024 03:00:00', 'Timestamp'] = '31/03/2024 01:00:00 - 31/03/2024 02:00:00'
        df.loc[df['Timestamp'] == '30/03/2025 01:00:00 - 30/03/2025 03:00:00', 'Timestamp'] = '30/03/2025 01:00:00 - 30/03/2025 02:00:00'

        df.loc[df['Timestamp'] == '27/10/2024 02:00:00 - 27/10/2024 02:00:00', 'Timestamp'] = '27/10/2024 02:00:00 - 27/10/2024 03:00:00'
        df.loc[df['Timestamp'] == '29/10/2023 02:00:00 - 29/10/2023 02:00:00', 'Timestamp'] = '29/10/2023 02:00:00 - 29/10/2023 03:00:00'

        
        new_timestamps = [
        ('26/03/2023 01:00:00 - 26/03/2023 02:00:00', '26/03/2023 02:00:00 - 26/03/2023 03:00:00'),
         ('31/03/2024 01:00:00 - 31/03/2024 02:00:00', '31/03/2024 02:00:00 - 31/03/2024 03:00:00'),
        ('30/03/2025 01:00:00 - 30/03/2025 02:00:00', '30/03/2025 02:00:00 - 30/03/2025 03:00:00'),
        ]

        new_rows = []
        for old_ts, new_ts in new_timestamps:
            match = df[df['Timestamp'] == old_ts]
            if not match.empty:
                new_row = match.copy()
                new_row['Timestamp'] = new_ts
                new_rows.append(new_row)

        if new_rows:
            df = pd.concat([df] + new_rows, ignore_index=True)

        df = df.groupby("Timestamp", as_index=False).agg({
            "Price": "mean"
            })
       
        df[["start", "end"]] = df["Timestamp"].str.split(" - ", expand=True)

        df["start_datetime"] = pd.to_datetime(df["start"], format="%d/%m/%Y  %H:%M:%S")
        df["end_datetime"] = pd.to_datetime(df["end"], format="%d/%m/%Y  %H:%M:%S")

        if year_in_file == "2025":
            end_of_april = pd.Timestamp("2025-04-30 23:00:00")
            df = df[df["start_datetime"] <= end_of_april]
    
        df = df.sort_values(by='start_datetime')

        cols = [
            "start_datetime", "end_datetime",
            "Timestamp", "Price"
        ]
        return df[cols]

    dfs = [load_and_process_excel(os.path.join(folder_path, f)) for f in file_names]
    return pd.concat(dfs, ignore_index=True)


def load_data():
    folder_path = "C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data"
    loadfile_names = [
        "Load_2023.xlsx",
        "Load_2024.xlsx",
        "Load_2025.xlsx",
    ]
    def load_and_process_excel(file_path):
        filename = os.path.basename(file_path)
        year_in_file = ''.join(filter(str.isdigit, filename))

        df = pd.read_excel(file_path, skiprows=6, names=["Timestamp", "Actual_Load", "Forecasted_Load"])
        df["Timestamp"] = df["Timestamp"].str.replace(r"\s*\(.*?\)", "", regex=True).str.strip()
        df[["start", "end"]] = df["Timestamp"].str.split(" - ", expand=True)
        df["start_datetime"] = pd.to_datetime(df["start"], format="%d/%m/%Y  %H:%M")
        df["end_datetime"] = pd.to_datetime(df["end"], format="%d/%m/%Y  %H:%M")
        

        if year_in_file == "2025":
            end_of_april = pd.Timestamp("2025-04-30 23:45")
            df = df[df["start_datetime"] <= end_of_april]

        df[["Actual_Load","Forecasted_Load"]] = df[["Actual_Load","Forecasted_Load"]].apply(pd.to_numeric, errors="coerce")

        df["start_datetime"] = (
            df["start_datetime"]
              .dt.tz_localize("Europe/Amsterdam", ambiguous="infer")
              .dt.tz_convert("UTC")
        )
        df["end_datetime"] = (
            df["end_datetime"]
              .dt.tz_localize("Europe/Amsterdam", ambiguous="infer")
              .dt.tz_convert("UTC")
        )

        df['hour_start'] = df['start_datetime'].dt.floor("h")
        df['hour_end']   = df['end_datetime'].dt.ceil("h")

        agg_df = (df.groupby(["hour_start", "hour_end"])
          .agg({"Actual_Load":"sum","Forecasted_Load":"sum"})
          .round(2)
          .reset_index()
    )
        agg_df["start_datetime"] = (
            agg_df["hour_start"]
            .dt.tz_convert("Europe/Amsterdam")
        )
        agg_df["end_datetime"] = (
            agg_df["hour_end"]  
            .dt.tz_convert("Europe/Amsterdam")
        )

        agg_df["Timestamp"] = agg_df["start_datetime"].dt.strftime("%d/%m/%Y %H:%M:%S") + " - " + agg_df["end_datetime"].dt.strftime("%d/%m/%Y %H:%M:%S")
        agg_df.loc[agg_df['Timestamp'] == '26/03/2023 01:00:00 - 26/03/2023 03:00:00', 'Timestamp'] = '26/03/2023 01:00:00 - 26/03/2023 02:00:00'
        agg_df.loc[agg_df['Timestamp'] == '31/03/2024 01:00:00 - 31/03/2024 03:00:00', 'Timestamp'] = '31/03/2024 01:00:00 - 31/03/2024 02:00:00'
        agg_df.loc[agg_df['Timestamp'] == '30/03/2025 01:00:00 - 30/03/2025 03:00:00', 'Timestamp'] = '30/03/2025 01:00:00 - 30/03/2025 02:00:00'

        agg_df.loc[agg_df['Timestamp'] == '27/10/2024 02:00:00 - 27/10/2024 02:00:00', 'Timestamp'] = '27/10/2024 02:00:00 - 27/10/2024 03:00:00'
        agg_df.loc[agg_df['Timestamp'] == '29/10/2023 02:00:00 - 29/10/2023 02:00:00', 'Timestamp'] = '29/10/2023 02:00:00 - 29/10/2023 03:00:00'

        
        new_timestamps = [
        ('26/03/2023 01:00:00 - 26/03/2023 02:00:00', '26/03/2023 02:00:00 - 26/03/2023 03:00:00'),
         ('31/03/2024 01:00:00 - 31/03/2024 02:00:00', '31/03/2024 02:00:00 - 31/03/2024 03:00:00'),
        ('30/03/2025 01:00:00 - 30/03/2025 02:00:00', '30/03/2025 02:00:00 - 30/03/2025 03:00:00'),
        ]
        new_rows = []
        for old_ts, new_ts in new_timestamps:
            match = agg_df[agg_df['Timestamp'] == old_ts]
            if not match.empty:
                new_row = match.copy()
                new_row['Timestamp'] = new_ts
                new_rows.append(new_row)

        if new_rows:
            agg_df = pd.concat([agg_df] + new_rows, ignore_index=True)

        agg_df = agg_df.groupby("Timestamp", as_index=False).agg({
            "Actual_Load": "mean",
            "Forecasted_Load": "mean"
            })
       
        agg_df[["start", "end"]] = agg_df["Timestamp"].str.split(" - ", expand=True)

        agg_df["start_datetime"] = pd.to_datetime(agg_df["start"], format="%d/%m/%Y  %H:%M:%S")
        agg_df["end_datetime"] = pd.to_datetime(agg_df["end"], format="%d/%m/%Y  %H:%M:%S")

    
        agg_df = agg_df.sort_values(by='start_datetime')

        return agg_df[[
            "start_datetime", "end_datetime",
             "Actual_Load", "Forecasted_Load"
        ]]
    
    dfs = [load_and_process_excel(os.path.join(folder_path, f)) for f in loadfile_names]
    return pd.concat(dfs, ignore_index=True)


comb_prices = price_data()
comb_load = load_data()
comb_prices.to_csv("C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/Combined_Prices.csv", index=False)
comb_load.to_csv("C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/Combined_Load_new.csv", index=False)

Use API from NED.nl to load in the Solar and Wind data for 2025

In [ ]:
import requests
import csv
from datetime import datetime, timedelta


API_KEY = ''
BASE_URL = "https://api.ned.nl/v1/utilizations"
HEADERS = {
    'X-AUTH-TOKEN': API_KEY,
    'accept': 'text/csv' 
}

START_DATE = datetime(2025, 1, 1)
END_DATE = datetime(2025, 5, 17)
STEP = timedelta(days=3)  

OUTPUT_PATH = "C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/WindOffShore_2025.csv"

header_written = False

current_start = START_DATE
while current_start < END_DATE:
    current_end = min(current_start + STEP, END_DATE)

    start_str = current_start.strftime("%Y-%m-%d")
    end_str = current_end.strftime("%Y-%m-%d")

    print(f"📡 Fetching from {start_str} to {end_str}...")

    params = {
        'point': 0,
        'type': 17,
        'granularity': 5,
        'granularitytimezone': 1,
        'classification': 2,
        'activity': 1,
        'validfrom[after]': start_str,
        'validfrom[strictly_before]': end_str
    }

    response = requests.get(BASE_URL, headers=HEADERS, params=params)

    if response.status_code == 200:
        lines = response.text.splitlines()
        if not lines:
            print(f" No data for {start_str} to {end_str}")
        else:
            with open(OUTPUT_PATH, mode='a', newline='') as file:
                writer = csv.writer(file)
                reader = csv.reader(lines)

                for i, row in enumerate(reader):
                    if i == 0 and not header_written:
                        row = [col.replace("validfrom", "validfrom (UTC)").replace("validto", "validto (UTC)").replace("volume", "volume (kWh)") for col in row]
                        writer.writerow(row)
                        header_written = True
                    elif i > 0:
                        writer.writerow(row)
    else:
        print(f" Failed for {start_str} to {end_str}: {response.status_code} {response.text}")

    current_start = current_end

print(f" Data retrieved and saved to: {OUTPUT_PATH}")

Combine 2023, 2024 and 2025 Solar and Wind data into a single DataFrame

In [ ]:
import pandas as pd
import os
import datetime

def load_exo(name):
    folder_path = "C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data"
    if name == "Solar":
        file_names = [
            "Solar_2023.csv",
            "Solar_2024.csv",
            "Solar_2025.csv"
        ]
    elif name == "WindShore":
        file_names = [
            "WindShore_2023.csv",
            "WindShore_2024.csv",
            "WindShore_2025.csv"
        ]
    elif name == "WindOffShore":
        file_names = [
            "WindOffShore_2023.csv",
            "WindOffShore_2024.csv",
            "WindOffShore_2025.csv"
        ]

    def load_and_process_excel(file_path, name):
        df = pd.read_csv(
            file_path,
            parse_dates=['validfrom (UTC)', 'validto (UTC)']
        )


        df = df.rename(columns={
            'validfrom (UTC)': 'start_datetime',
            'validto (UTC)'  : 'end_datetime',
            'volume (kWh)'   : name  
        })
        if df["start_datetime"].dt.tz is None:
            df["start_datetime"] = (
                df["start_datetime"]
                .dt.tz_localize("UTC", ambiguous="infer")
                .dt.tz_convert("Europe/Amsterdam")
                .dt.tz_localize(None))
        else:
            df["start_datetime"] = (
                df["start_datetime"]
                .dt.tz_convert("Europe/Amsterdam")
                .dt.tz_localize(None))
            
        if df["end_datetime"].dt.tz is None:
            df["end_datetime"] = (
                df["end_datetime"]
                .dt.tz_localize("UTC", ambiguous="infer")
                .dt.tz_convert("Europe/Amsterdam")
                .dt.tz_localize(None))
        else:
            df["end_datetime"] = (
                df["end_datetime"]
                .dt.tz_convert("Europe/Amsterdam")
                .dt.tz_localize(None))
        
        end_of_april = pd.Timestamp("2025-04-30 23:00:00")
        df = df[df["start_datetime"] <= end_of_april]

        df["Timestamp"] = df["start_datetime"].dt.strftime("%d/%m/%Y %H:%M:%S") + " - " + df["end_datetime"].dt.strftime("%d/%m/%Y %H:%M:%S")
        df.loc[df['Timestamp'] == '26/03/2023 01:00:00 - 26/03/2023 03:00:00', 'Timestamp'] = '26/03/2023 01:00:00 - 26/03/2023 02:00:00'
        df.loc[df['Timestamp'] == '31/03/2024 01:00:00 - 31/03/2024 03:00:00', 'Timestamp'] = '31/03/2024 01:00:00 - 31/03/2024 02:00:00'
        df.loc[df['Timestamp'] == '30/03/2025 01:00:00 - 30/03/2025 03:00:00', 'Timestamp'] = '30/03/2025 01:00:00 - 30/03/2025 02:00:00'

        df.loc[df['Timestamp'] == '27/10/2024 02:00:00 - 27/10/2024 02:00:00', 'Timestamp'] = '27/10/2024 02:00:00 - 27/10/2024 03:00:00'
        df.loc[df['Timestamp'] == '29/10/2023 02:00:00 - 29/10/2023 02:00:00', 'Timestamp'] = '29/10/2023 02:00:00 - 29/10/2023 03:00:00'

        
        new_timestamps = [
        ('26/03/2023 01:00:00 - 26/03/2023 02:00:00', '26/03/2023 02:00:00 - 26/03/2023 03:00:00'),
         ('31/03/2024 01:00:00 - 31/03/2024 02:00:00', '31/03/2024 02:00:00 - 31/03/2024 03:00:00'),
        ('30/03/2025 01:00:00 - 30/03/2025 02:00:00', '30/03/2025 02:00:00 - 30/03/2025 03:00:00'),
        ]

        new_rows = []
        for old_ts, new_ts in new_timestamps:
            match = df[df['Timestamp'] == old_ts]
            if not match.empty:
                new_row = match.copy()
                new_row['Timestamp'] = new_ts
                new_rows.append(new_row)

        if new_rows:
            df = pd.concat([df] + new_rows, ignore_index=True)

        df = df.groupby("Timestamp", as_index=False).agg({
            name: "mean"
            })
       
        df[["start", "end"]] = df["Timestamp"].str.split(" - ", expand=True)

        df["start_datetime"] = pd.to_datetime(df["start"], format="%d/%m/%Y  %H:%M:%S")
        df["end_datetime"] = pd.to_datetime(df["end"], format="%d/%m/%Y  %H:%M:%S")

        df = df.sort_values(by='start_datetime')

        return df[['start_datetime', 'end_datetime', name]]

    dfs = [load_and_process_excel(os.path.join(folder_path, f), name) for f in file_names]
    return pd.concat(dfs, ignore_index=True)

Solar = load_exo("Solar")
Solar.to_csv("C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/Combined_Solar.csv", index=False)

WindShore = load_exo("WindShore")  
WindShore.to_csv("C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/Combined_WindShore.csv", index=False)  

WindOffShore = load_exo("WindOffShore")
WindOffShore.to_csv("C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/Combined_WindOffShore.csv", index=False)


Combine the Price, Load, Solar and Wind datasets

In [ ]:
import pandas as pd
import os

folder_path = "C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data"
files = {
    "Prices": "Combined_Prices.csv",
    "Load": "Combined_Load_new.csv",
    "Solar": "Combined_Solar.csv",
    "WindShore": "Combined_WindShore.csv",
    "WindOffShore": "Combined_WindOffShore.csv"
}

def load_csv(file_name):
    df = pd.read_csv(
        os.path.join(folder_path, file_name),
        parse_dates=['start_datetime', 'end_datetime']
    )
    return df

df_prices = load_csv(files["Prices"])
df_load = load_csv(files["Load"])
df_solar = load_csv(files["Solar"])
df_wind_shore = load_csv(files["WindShore"])
df_wind_offshore = load_csv(files["WindOffShore"])

data = df_prices \
    .merge(df_load, on=['start_datetime', 'end_datetime'], how='outer') \
    .merge(df_solar, on=['start_datetime', 'end_datetime'], how='outer') \
    .merge(df_wind_shore, on=['start_datetime', 'end_datetime'], how='outer') \
    .merge(df_wind_offshore, on=['start_datetime', 'end_datetime'], how='outer')


data = data.sort_values(by='start_datetime').reset_index(drop=True)
data['Solar'] = data['Solar']/1000
data['WindShore'] = data['WindShore']/1000
data['WindOffShore'] = data['WindOffShore']/1000
data['Load'] = data['Actual_Load'] /4
data = data.drop("Actual_Load", axis='columns')
data = data.drop("Forecasted_Load", axis='columns')
data.to_csv(os.path.join(folder_path, "All_Combined_new.csv"), index=False)

print(" Merged dataset saved as Combined_All_Features_new.csv")


Add Import and Export to the DataFrame

In [ ]:
import pandas as pd

Import_export_2023 = pd.read_csv(
    "C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/Import_export_2023.csv",
    sep=';',
    parse_dates=["start_datetime", "end_datetime"], dayfirst=True
)

Import_export_2024 = pd.read_csv(
    "C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/Import_export_2024.csv",
    sep=';',
    parse_dates=["start_datetime", "end_datetime"], dayfirst=True
)

Import_export_2025 = pd.read_csv(
    "C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/Import_export_2025.csv",
    sep=';',
    parse_dates=["start_datetime", "end_datetime"], dayfirst=True
)

Import_export = pd.concat([Import_export_2023, Import_export_2024, Import_export_2025])
Import_export = Import_export.replace("n/e", pd.NA).dropna()
Import_export['Physical Flow (MW)'] = pd.to_numeric(Import_export['Physical Flow (MW)'], errors='coerce')
Import_export = Import_export.dropna()


Export = Import_export.groupby(['start_datetime', 'Out Area']).agg({
    'Physical Flow (MW)': 'sum'
}).reset_index()

Import = Import_export.groupby(['start_datetime', 'In Area']).agg({
    'Physical Flow (MW)': 'sum'
}).reset_index()

Export = Export[Export['Out Area'] == 'NL'].drop(columns='Out Area')
Import = Import[Import['In Area'] == 'NL'].drop(columns= 'In Area')
Export.columns = ['start_datetime', 'Export']
Import.columns = ['start_datetime', 'Import']
Import_export = Export.merge(Import, how='outer', on = 'start_datetime')
Import_export['Net_Import'] = (Import_export['Import'] - Import_export['Export']).round(2)
Import_export = Import_export.drop(['Import', 'Export'], axis='columns')

df = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv', parse_dates=['start_datetime', 'end_datetime'])
df_added = df.merge(Import_export, how='left', on='start_datetime')
df_added = df_added.fillna(0)

df_added.to_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv', index=False)


Add EUA, API2 Coal and TTF Gas Prices to the DataFrame

In [ ]:
import pandas as pd

EUA_price = pd.read_excel(
    "C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/EUA_price.xlsx", parse_dates=["Exchange Date"]
)

Coal_price = pd.read_csv(
    "C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/Coal_price.csv",
    parse_dates=["Date"], dayfirst=False
)

Gas_price = pd.read_csv(
    "C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/Gas_price.csv",
    parse_dates=["Date"], dayfirst=False
)

EUA_price['Date'] = EUA_price['Exchange Date'].dt.date
Coal_price['Date'] = Coal_price['Date'].dt.date
Gas_price['Date'] = Gas_price['Date'].dt.date

EUA_price['EUA_Price'] = EUA_price['Trade Price']
Coal_price['Coal_Price'] = (Coal_price['Price'] * 0.86).round(2)
Gas_price['Gas_Price'] = Gas_price['Price']

Coal_price = Coal_price.drop(['Open', 'High', 'Low', 'Vol.', 'Change %', 'Price'], axis = 'columns')
Gas_price = Gas_price.drop(['Open', 'High', 'Low', 'Vol.', 'Change %', 'Price'], axis = 'columns')
EUA_price = EUA_price.drop(['Exchange Date', 'Trade Price'], axis='columns')


all_combined_new = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv', parse_dates=['start_datetime', 'end_datetime'])
all_combined_new['Date'] = all_combined_new['start_datetime'].dt.date

all_combined_new = all_combined_new.merge(EUA_price, how='left', on='Date')
all_combined_new = all_combined_new.merge(Coal_price, how='left', on='Date')
all_combined_new = all_combined_new.merge(Gas_price, how='left', on='Date')

all_combined_new = all_combined_new.fillna(method='ffill')
all_combined_new = all_combined_new.fillna(method='bfill')
all_combined_new = all_combined_new.drop('Date', axis='columns')

all_combined_new.to_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv', index=False)


Determine which features are important for price forecasting through correlation, VIF and Lasso

In [ ]:
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import seaborn as sns
import matplotlib.pyplot as plt

all_combined_new = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv', parse_dates=['start_datetime', 'end_datetime'])
all_combined_new.dropna(inplace=True)

train_df = all_combined_new[(all_combined_new['start_datetime'].dt.year == 2023)]
for lag in [72, 168]:
    train_df[f'Price_lag_{lag}'] = train_df['Price'].shift(lag)
train_df.dropna(inplace=True)

feature_cols = ['Solar', 'WindShore', 'WindOffShore','Load', 'Net_Import','EUA_Price', 'Coal_Price', 'Gas_Price', 'Price_lag_72', 'Price_lag_168']

correlation_matrix = train_df.corr(numeric_only=True)


plt.figure(figsize=(9, 5))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.savefig("C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Output/Plot/correlation_matrix.png", format="png", bbox_inches='tight')

plt.show()

X_train = train_df[feature_cols] 
y_train = train_df['Price']


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)
lasso = LassoCV(cv=10).fit(X_train, y_train)
print(lasso)

selected = [feature for coef, feature in zip(lasso.coef_, feature_cols) if coef != 0]
print("Selected features:", selected)

coef = pd.Series(lasso.coef_, index=X_train.columns)
intercept = lasso.intercept_
alpha = lasso.alpha_

print("\nLasso Regression Results")
print("-" * 30)
print(f"Intercept: {intercept:.4f}")
print(f"Chosen alpha (regularization strength): {alpha:.6f}")
print("\nCoefficients:")
print(coef)


X = add_constant(train_df[feature_cols])
vif_df = pd.DataFrame()
vif_df["Feature"] = X.columns
vif_df["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_df)

newcolumns = ['Solar', 'WindShore', 'WindOffShore','Load', 'Net_Import', 'Coal_Price','Price_lag_72', 'Price_lag_168']
X = add_constant(train_df[newcolumns])
vif_df = pd.DataFrame()
vif_df["Feature"] = X.columns
vif_df["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_df)



Generate summary statistics for the chosen variables

In [ ]:
import pandas as pd

df = pd.read_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/All_Combined_new.csv', parse_dates=['start_datetime', 'end_datetime'])
df.dropna(inplace=True)

train_df = df[(df['start_datetime'].dt.year == 2023)]
val_df = df[((df['start_datetime'].dt.year == 2024)& (df['start_datetime'].dt.month <= 6))]
test_df  = df[((df['start_datetime'].dt.year == 2024) & (df['start_datetime'].dt.month >= 7)) | (df['start_datetime'].dt.year == 2025)]
feature_cols = ['Load', 'Solar', 'WindShore', 'WindOffShore', 'Net_Import', 'Coal_Price']


summary_stats_train = train_df.describe()
print(summary_stats_train)

summary_stats_val = val_df.describe()
print(summary_stats_val)

summary_start_test = test_df.describe()
print(summary_start_test)

summary_stats_all = df.describe()
print(summary_stats_all)




Use API from Ned.nl to get the Electricity prices for 2025 to determine the noise to add to our variables. 

In [ ]:
import requests
import csv
from datetime import datetime, timedelta


API_KEY = ''
BASE_URL = "https://api.ned.nl/v1/utilizations"
HEADERS = {
    'X-AUTH-TOKEN': API_KEY,
    'accept': 'text/csv' 
}

START_DATE = datetime(2025, 3, 1)
END_DATE = datetime(2025, 6, 30)
STEP = timedelta(days=3) 


OUTPUT_PATH = "C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/SolarMar_Jun.csv"
header_written = False

current_start = START_DATE
while current_start < END_DATE:
    current_end = min(current_start + STEP, END_DATE)

    start_str = current_start.strftime("%Y-%m-%d")
    end_str = current_end.strftime("%Y-%m-%d")

    print(f"📡 Fetching from {start_str} to {end_str}...")

    params = {
        'point': 0,
        'type': 2,
        'granularity': 5,
        'granularitytimezone': 1,
        'classification': 2,
        'activity': 1,
        'validfrom[after]': start_str,
        'validfrom[strictly_before]': end_str
    }

    response = requests.get(BASE_URL, headers=HEADERS, params=params)

    if response.status_code == 200:
        lines = response.text.splitlines()
        if not lines:
            print(f" No data for {start_str} to {end_str}")
        else:
            with open(OUTPUT_PATH, mode='a', newline='') as file:
                writer = csv.writer(file)
                reader = csv.reader(lines)

                for i, row in enumerate(reader):
                    if i == 0 and not header_written:
                        row = [col.replace("validfrom", "validfrom (UTC)").replace("validto", "validto (UTC)").replace("volume", "volume (kWh)") for col in row]
                        writer.writerow(row)
                        header_written = True
                    elif i > 0:
                        writer.writerow(row)
    else:
        print(f"Failed for {start_str} to {end_str}: {response.status_code} {response.text}")

    current_start = current_end

print(f" Data retrieved and saved to: {OUTPUT_PATH}")

Determining forecast error for solar and wind

In [ ]:
import pandas as pd
import os
import datetime
import numpy as np

def load_exo(name):
    folder_path = "C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar"
    if name == "Solar":
        file_names = [os.path.join("Zon", f"Zon_ ({i}).csv") for i in range(1, 19)]
        data_name = "SolarMar_Jun.csv"

    elif name == "WindShore":
        file_names = [os.path.join("Wind", f"Wind_ ({i}).csv") for i in range(1, 19)]
        data_name = "WindMar_Jun.csv"

    elif name == "WindOffShore":
        file_names = [os.path.join("Zeewind", f"Zeewind_ ({i}).csv") for i in range(1, 19)]
        data_name = "WindOffMar_Jun.csv"

    data = pd.read_csv(
        os.path.join(folder_path, data_name), sep=',',
        parse_dates=['validfrom (UTC)', 'validto (UTC)']
        )
    
    data = data.rename(columns={
            'validfrom (UTC)': 'start_datetime',
            'validto (UTC)'  : 'end_datetime',
            'volume (kWh)'   : f'{name}_actual'  
        })
    
    if data["start_datetime"].dt.tz is None:
        data["start_datetime"] = (
                data["start_datetime"]
                .dt.tz_localize("UTC", ambiguous="infer")
                .dt.tz_convert("Europe/Amsterdam")
                .dt.tz_localize(None))
    else:
        data["start_datetime"] = (
                data["start_datetime"]
                .dt.tz_convert("Europe/Amsterdam")
                .dt.tz_localize(None))
                
        
    if data["end_datetime"].dt.tz is None:
            data["end_datetime"] = (
                data["end_datetime"]
                .dt.tz_localize("UTC", ambiguous="infer")
                .dt.tz_convert("Europe/Amsterdam")
                .dt.tz_localize(None))
    else:
            data["end_datetime"] = (
                data["end_datetime"]
                .dt.tz_convert("Europe/Amsterdam")
                .dt.tz_localize(None))
    
    data = data[['start_datetime', 'end_datetime', f'{name}_actual']]

    def load_and_process_excel(file_path, name, data):
        df = pd.read_csv(
            file_path,
            parse_dates=['validfrom (UTC)', 'validto (UTC)']
        )

        df = df.rename(columns={
            'validfrom (UTC)': 'start_datetime',
            'validto (UTC)'  : 'end_datetime',
            'volume (kWh)'   : name   
        })
    
        if df["start_datetime"].dt.tz is None:
            df["start_datetime"] = (
                df["start_datetime"]
                .dt.tz_localize("UTC", ambiguous="infer")
                .dt.tz_convert("Europe/Amsterdam")
                .dt.tz_localize(None))
        else:
            df["start_datetime"] = (
                df["start_datetime"]
                .dt.tz_convert("Europe/Amsterdam")
                .dt.tz_localize(None))
            
        if df["end_datetime"].dt.tz is None:
            df["end_datetime"] = (
                df["end_datetime"]
                .dt.tz_localize("UTC", ambiguous="infer")
                .dt.tz_convert("Europe/Amsterdam")
                .dt.tz_localize(None))
        else:
            df["end_datetime"] = (
                df["end_datetime"]
                .dt.tz_convert("Europe/Amsterdam")
                .dt.tz_localize(None))
        
        df = df[['start_datetime', 'end_datetime', name]]
        
        full = df.merge(data, on = ["start_datetime", "end_datetime"], how = 'left', validate='one_to_one', sort=True)
        difference = full[name] - full[f'{name}_actual']

        return difference, full

    result = [load_and_process_excel(os.path.join(folder_path, f), name, data) for f in file_names]
    diffs, full = zip(*result)
    diffs_df = pd.concat(diffs, axis=1)
    full_df = pd.concat(full, axis=1)
    actuals_list = [f[[f"{name}_actual"]] for f in full]
    actuals_df   = pd.concat(actuals_list, axis=1) 
    pct_error_df = diffs_df.div(actuals_df.values)  
    
    return pct_error_df

def trimmed_row_mean_one_each(row):
    values = row.dropna().values
    values = values[np.isfinite(values)]

    if len(values) <= 4:
        return np.nan

    sorted_values = np.sort(values)
    trimmed = sorted_values[2:-2]

    return trimmed.mean() if len(trimmed) > 0 else np.nan

def trimmed_row_mean(df):
    return df.apply(trimmed_row_mean_one_each, axis=1)
    
def every_24_rows(s):
    return s.groupby(s.index // 24).mean(), s.groupby(s.index // 24).std(ddof=1)


Solar_dif = load_exo("Solar")
Solar_mu, Solar_Sigma = every_24_rows(trimmed_row_mean(Solar_dif))
Solar_diff = pd.concat([Solar_mu, Solar_Sigma], axis=1)
Solar_diff.columns = ['Solar_mu', 'Solar_sigma']
Solar_diff.to_csv("C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/Solar_error.csv", index=False)

WindShore = load_exo("WindShore")  
Windshore_mu, Winshore_Sigma= every_24_rows(trimmed_row_mean(WindShore))
WindShore_diff = pd.concat([Windshore_mu, Winshore_Sigma], axis=1)
WindShore_diff.columns = ['WindShore_mu', 'WindShore_sigma']
WindShore_diff.to_csv("C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindShore_error.csv", index=False)

WindOffShore = load_exo("WindOffShore")
WindOffshore_mu, WindOffShore_Sigma = every_24_rows(trimmed_row_mean(WindOffShore))
WindOffShore_diff = pd.concat([WindOffshore_mu, WindOffShore_Sigma], axis=1)
WindOffShore_diff.columns = ['WindOffShore_mu', 'WindOffShore_sigma']
WindOffShore_diff.to_csv("C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/WindOffShore_error.csv", index=False)


Determining forecast error for load

In [ ]:
import pandas as pd

df = pd.read_csv("C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/Load_Forecast_2023.csv", sep=';', parse_dates=['start_datetime', 'end_datetime'])


df['start_datetime'] = pd.to_datetime(
    df['start_datetime'],
    dayfirst=True,
    errors='raise'   
)
df['end_datetime'] = pd.to_datetime(
    df['end_datetime'],
    dayfirst=True,
    errors='raise'
)

df['start_datetime'] = df['start_datetime'].dt.floor('H')

hourly = df.groupby('start_datetime').agg({
    'Actual'   : 'sum',   
    'Forecast' : 'sum',  
}).reset_index()

hourly['Actual'] = hourly['Actual']/4
hourly['Forecast'] = hourly['Forecast']/4
hourly['end_datetime'] = hourly['start_datetime'] + pd.Timedelta(hours=1)
hourly['difference'] = hourly['Actual'] - hourly['Forecast']
hourly['pct_error'] = hourly['difference'] / hourly['Actual'].replace(0, pd.NA)  
mean_error = hourly['pct_error'].mean()
std_error = hourly['pct_error'].std()
load_error = pd.Series(
    {'load_mu':    mean_error,
     'load_sigma': std_error}
)
load_error.to_csv("C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data_wind_solar/Load_error.csv", index=False)


Generate the exact dates in the train, validation and test set

In [ ]:
import pandas as pd

train_dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
val_dates = pd.date_range(start='2024-01-01', end='2024-06-30', freq='D')
test_dates = pd.date_range(start='2024-07-01', end='2025-04-30', freq='D')

train_dates_df = pd.DataFrame(train_dates, columns=['date'])
val_dates_df = pd.DataFrame(val_dates, columns=['date'])
test_dates_df = pd.DataFrame(test_dates, columns=['date'])

train_dates_final = train_dates_df.iloc[7:-7]
val_dates_final = val_dates_df.iloc[7:-7]
test_dates_final = test_dates_df.iloc[7:-7]

train_dates_final.to_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/train_dates.csv', index=False)
val_dates_final.to_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/val_dates.csv', index=False)        
test_dates_final.to_csv('C:/Users/daniq/iCloudDrive/Erasmus University Rotterdam/Master/Thesis/Data/test_dates.csv', index=False)


